In [1]:
import numpy as np
np.random.seed(0)

def sign_pm1(x):
    """Sign function returning -1 or +1."""
    return np.where(x >= 0, 1, -1)



In [2]:
# ----------------------------------------------------------------------
# Q1: 10x10 associative memory using Hopfield network
# ----------------------------------------------------------------------

def bin_to_pm1(pattern_2d):
    """
    Convert 10x10 pattern of 0/1 to 1D vector of length 100 with values in {-1, +1}.
    """
    return 2 * pattern_2d.flatten() - 1

def pm1_to_bin(pattern_1d):
    """
    Convert {-1, +1} vector back to 0/1 10x10 pattern.
    """
    return ((pattern_1d + 1) // 2).reshape(10, 10).astype(int)

def train_hopfield(patterns_pm1):
    """
    Train a classic Hopfield network using Hebbian learning.

    patterns_pm1: list of 1D numpy arrays (length N=100), values in {-1, +1}.
    """
    n = patterns_pm1[0].size
    W = np.zeros((n, n))
    for p in patterns_pm1:
        W += np.outer(p, p)
    np.fill_diagonal(W, 0.0)
    W /= n
    return W

def recall_async(W, init_state, n_steps=2000):
    """
    Asynchronous update rule to recall pattern from initial state.

    W: (N x N) weight matrix.
    init_state: 1D numpy array in {-1, +1}.
    """
    s = init_state.copy()
    n = s.size
    for _ in range(n_steps):
        i = np.random.randint(0, n)
        h_i = np.dot(W[i], s)
        s[i] = 1 if h_i >= 0 else -1
    return s

def demo_q1():
    print("\n=== Q1: 10x10 Hopfield associative memory demo ===")

    # Define a few simple 10x10 binary patterns (0/1)
    pattern_A = np.zeros((10, 10), dtype=int)
    np.fill_diagonal(pattern_A, 1)  # A: main diagonal

    pattern_B = np.zeros((10, 10), dtype=int)
    pattern_B[0, :] = 1             # top row
    pattern_B[-1, :] = 1            # bottom row

    pattern_C = np.zeros((10, 10), dtype=int)
    pattern_C[:, 0] = 1             # left column
    pattern_C[:, -1] = 1            # right column

    patterns_bin = [pattern_A, pattern_B, pattern_C]
    patterns_pm1 = [bin_to_pm1(p) for p in patterns_bin]

    W = train_hopfield(patterns_pm1)

    # Take pattern_A, corrupt it, and recall
    original = patterns_pm1[0]
    corrupted = original.copy()
    flip_indices = np.random.choice(len(corrupted), size=20, replace=False)
    corrupted[flip_indices] *= -1

    recovered = recall_async(W, corrupted, n_steps=3000)

    print("Original pattern A (0/1):")
    print(pm1_to_bin(original))
    print("Corrupted version (0/1):")
    print(pm1_to_bin(corrupted))
    print("Recovered version (0/1):")
    print(pm1_to_bin(recovered))

demo_q1()


=== Q1: 10x10 Hopfield associative memory demo ===
Original pattern A (0/1):
[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]
Corrupted version (0/1):
[[1 0 1 0 0 0 0 1 1 0]
 [0 1 0 1 0 0 1 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 1 0 1 0]
 [0 0 1 1 0 1 0 0 0 1]]
Recovered version (0/1):
[[1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1]]


In [3]:
# ----------------------------------------------------------------------
# Q2: Capacity of Hopfield network
# ----------------------------------------------------------------------

def hopfield_capacity(N):
    """
    Theoretical approximate capacity of Hopfield net with N neurons:
    P_max ≈ 0.138 * N (for random uncorrelated patterns).
    """
    return 0.138 * N

def demo_q2():
    print("\n=== Q2: Capacity of Hopfield network ===")
    N = 10 * 10
    cap = hopfield_capacity(N)
    print(f"For N = {N} neurons, approximate capacity ≈ 0.138 * {N} ≈ {cap:.2f}")
    print(f"≈ {int(cap)} patterns (for random, uncorrelated patterns).")
demo_q2()


=== Q2: Capacity of Hopfield network ===
For N = 100 neurons, approximate capacity ≈ 0.138 * 100 ≈ 13.80
≈ 13 patterns (for random, uncorrelated patterns).


In [4]:
# ----------------------------------------------------------------------
# Q3: Error-correcting capability (experiment)
# ----------------------------------------------------------------------

def flip_k_bits_pm1(pattern_pm1, k):
    """
    Flip (multiply by -1) exactly k random bits in pattern_pm1.
    """
    p = pattern_pm1.copy()
    idx = np.random.choice(len(p), size=k, replace=False)
    p[idx] *= -1
    return p

def estimate_error_correction(W, stored_patterns_pm1, k, trials=50):
    """
    Estimate probability that network corrects k flipped bits.

    For each trial:
      - pick random stored pattern
      - flip k bits
      - run recall
      - check if recovered == original
    """
    n_correct = 0
    for _ in range(trials):
        p = stored_patterns_pm1[np.random.randint(len(stored_patterns_pm1))]
        corrupted = flip_k_bits_pm1(p, k)
        recovered = recall_async(W, corrupted, n_steps=2000)
        if np.array_equal(recovered, p):
            n_correct += 1
    return n_correct / trials

def demo_q3():
    print("\n=== Q3: Error-correcting capability experiment ===")
    # Reuse patterns from Q1 for consistency
    pattern_A = np.zeros((10, 10), dtype=int)
    np.fill_diagonal(pattern_A, 1)
    pattern_B = np.zeros((10, 10), dtype=int)
    pattern_B[0, :] = 1
    pattern_B[-1, :] = 1
    pattern_C = np.zeros((10, 10), dtype=int)
    pattern_C[:, 0] = 1
    pattern_C[:, -1] = 1

    patterns_bin = [pattern_A, pattern_B, pattern_C]
    patterns_pm1 = [bin_to_pm1(p) for p in patterns_bin]

    W = train_hopfield(patterns_pm1)

    # Try different values of k (number of flipped bits)
    for k in [0, 5, 10, 15, 20, 25, 30]:
        acc = estimate_error_correction(W, patterns_pm1, k, trials=20)
        print(f"k = {k} flipped bits -> success rate ≈ {acc*100:.1f}%")

    print("Observation: as k increases, success rate decreases.")

demo_q3()


=== Q3: Error-correcting capability experiment ===
k = 0 flipped bits -> success rate ≈ 100.0%
k = 5 flipped bits -> success rate ≈ 70.0%
k = 10 flipped bits -> success rate ≈ 65.0%
k = 15 flipped bits -> success rate ≈ 80.0%
k = 20 flipped bits -> success rate ≈ 65.0%
k = 25 flipped bits -> success rate ≈ 25.0%
k = 30 flipped bits -> success rate ≈ 35.0%
Observation: as k increases, success rate decreases.


In [5]:
# ----------------------------------------------------------------------
# Q4: Eight-rook problem using Hopfield-style energy minimization
# ----------------------------------------------------------------------

def rook_energy(board):
    """
    Energy for eight-rook problem.

    board: 8x8 array with 0/1 entries (1 = rook present).

    E = A * sum_rows (row_sum - 1)^2 + B * sum_cols (col_sum - 1)^2
    """
    A = 1.0
    B = 1.0
    row_sums = board.sum(axis=1)
    col_sums = board.sum(axis=0)
    E_row = A * np.sum((row_sums - 1) ** 2)
    E_col = B * np.sum((col_sums - 1) ** 2)
    return E_row + E_col

def solve_eight_rooks(max_iters=10000):
    """
    Use greedy energy-decreasing flips (Hopfield-style search)
    to solve the eight-rook problem.
    """
    # Random initial 8x8 board of 0/1
    board = (np.random.rand(8, 8) > 0.5).astype(int)
    E = rook_energy(board)

    for it in range(max_iters):
        improved = False
        # Try flipping each cell once per iteration
        for i in range(8):
            for j in range(8):
                board[i, j] ^= 1  # flip 0->1 or 1->0
                E_new = rook_energy(board)
                if E_new <= E:
                    # keep flip if energy doesn't increase
                    E = E_new
                    improved = True
                else:
                    # revert flip
                    board[i, j] ^= 1

        if not improved:  # local minimum
            break

    return board, E

def demo_q4():
    print("\n=== Q4: Eight-rook problem using Hopfield-style energy minimization ===")
    solution, E_final = solve_eight_rooks(max_iters=20000)
    print("Final board configuration (1 = rook, 0 = empty):")
    print(solution)
    print("Row sums:", solution.sum(axis=1))
    print("Col sums:", solution.sum(axis=0))
    print("Final energy:", E_final)
    if np.all(solution.sum(axis=1) == 1) and np.all(solution.sum(axis=0) == 1):
        print("=> Valid non-attacking eight-rook configuration found.")
    else:
        print("=> Local minimum (might not be perfect). Rerun to try again.")

demo_q4()


=== Q4: Eight-rook problem using Hopfield-style energy minimization ===
Final board configuration (1 = rook, 0 = empty):
[[0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]]
Row sums: [1 1 1 1 2 1 1 1]
Col sums: [1 1 1 2 1 1 1 1]
Final energy: 2.0
=> Local minimum (might not be perfect). Rerun to try again.


In [6]:
# ----------------------------------------------------------------------
# Q5: TSP (10 cities) with Hopfield–Tank network
# ----------------------------------------------------------------------

def hopfield_tsp(dist_matrix,
                 A=500.0, B=500.0, C=200.0,
                 eta=0.01, n_steps=5000):
    """
    Hopfield–Tank style continuous network for TSP.

    dist_matrix: n x n symmetric distance matrix. n=10 for this lab.

    V[x, i] ≈ probability/neuron output that city x is visited at position i.
    """
    n = dist_matrix.shape[0]
    U = np.random.randn(n, n)   # internal states
    V = 0.5 * (1 + np.tanh(U))  # outputs in (0,1)

    for step in range(n_steps):
        dE_dV = np.zeros((n, n))

        # Constraint 1: each city appears exactly once
        for x in range(n):
            row_sum = V[x, :].sum()
            dE_dV[x, :] += 2 * A * (row_sum - 1)

        # Constraint 2: each position has exactly one city
        for i in range(n):
            col_sum = V[:, i].sum()
            dE_dV[:, i] += 2 * B * (col_sum - 1)

        # Distance term: tour length
        for i in range(n):
            ip1 = (i + 1) % n
            # For each pair of cities (x,y) at positions i and i+1
            for x in range(n):
                dE_dV[x, i] += C * np.sum(dist_matrix[x, :] * V[:, ip1])

        # Gradient descent on U
        U -= eta * dE_dV
        V = 0.5 * (1 + np.tanh(U))

    return V

def decode_tour(V):
    """
    Given final V[x, i], decode a discrete tour:
    For each position i, pick the city x with maximum V[x, i].
    """
    return V.argmax(axis=0)

def tsp_total_length(tour, dist_matrix):
    """
    Compute total length of a cyclic tour order.
    tour: length-n array of city indices.
    """
    n = len(tour)
    length = 0.0
    for i in range(n):
        x = tour[i]
        y = tour[(i + 1) % n]
        length += dist_matrix[x, y]
    return length

def demo_q5():
    print("\n=== Q5: 10-city TSP using Hopfield–Tank network ===")

    n_cities = 10
    N_neurons = n_cities * n_cities
    # Number of weights in fully connected symmetric Hopfield net:
    num_weights = N_neurons * (N_neurons - 1) // 2
    print(f"Number of neurons: N = {N_neurons}")
    print(f"Number of distinct weights in fully connected symmetric network: {num_weights}")

    # Example: random symmetric distance matrix with zero diagonal
    D = np.random.rand(n_cities, n_cities)
    D = (D + D.T) / 2
    np.fill_diagonal(D, 0.0)

    print("Distance matrix (10x10):")
    print(np.round(D, 2))

    V_final = hopfield_tsp(D, n_steps=4000)
    tour = decode_tour(V_final)
    length = tsp_total_length(tour, D)

    print("Decoded tour (city indices 0..9):")
    print(tour)
    print(f"Total tour length: {length:.3f}")

demo_q5()


=== Q5: 10-city TSP using Hopfield–Tank network ===
Number of neurons: N = 100
Number of distinct weights in fully connected symmetric network: 4950
Distance matrix (10x10):
[[0.   0.36 0.36 0.27 0.56 0.47 0.51 0.58 0.61 0.55]
 [0.36 0.   0.61 0.73 0.28 0.27 0.4  0.63 0.81 0.44]
 [0.36 0.61 0.   0.35 0.27 0.49 0.49 0.53 0.32 0.59]
 [0.27 0.73 0.35 0.   0.31 0.59 0.06 0.22 0.49 0.94]
 [0.56 0.28 0.27 0.31 0.   0.45 0.4  0.51 0.31 0.48]
 [0.47 0.27 0.49 0.59 0.45 0.   0.48 0.95 0.16 0.31]
 [0.51 0.4  0.49 0.06 0.4  0.48 0.   0.46 0.19 0.29]
 [0.58 0.63 0.53 0.22 0.51 0.95 0.46 0.   0.31 0.38]
 [0.61 0.81 0.32 0.49 0.31 0.16 0.19 0.31 0.   0.45]
 [0.55 0.44 0.59 0.94 0.48 0.31 0.29 0.38 0.45 0.  ]]
Decoded tour (city indices 0..9):
[7 3 6 8 5 0 1 4 2 7]
Total tour length: 2.534
